In [1]:
# CODE is based of fastautoagument code here 

# https://github.com/kakaobrain/fast-autoaugment/blob/master/FastAutoAugment/search.py

import torch 
import torch.nn as nn
import ray 

import ray
from ray import tune
from ray.tune import track

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch

from ray.tune import register_trainable, run_experiments
import wandb
import argparse
import torchvision.models as models
import sys

sys.path.append("/userdata/smetzger/all_deepul_files/deepul_proj/moco/")
import moco.loader
import moco.builder
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))
import os
import ray.tune as tune
# What to use for our moco model. 

# argparser.add('--checkpoint_fp', type=str, help='base file path where everything is stored.')
# argparser.add('-checkpoints' , '--list', nargs='+', help='The list of the checkpoint codes, in order by cv fold')
# # example: search.py -checkpoints rdElg fxrZE IJu2W vnhKs esdq2

# argparser.add('--data', type=str, help="Where the data directory is")
# argparser.add('--dataid', type=str, default='cifar10', help="imagenet or cifar")
 
# # Arguments for FAA.  
# parser.add_argument('--num-op', type=int, default=2, help="number of operations per subpolicy.")
# parser.add_argument('--num-policy', type=int, default=5, help="number of subpolicies in each policy")
# parser.add_argument('--num-search', type=int, default=200, help="number of hyperopt iterations to run.")
# parser.add_argument('--smoke-test', action='store_true', help="quick test of our search")
# args = parser.parse_args()


# FOR DEBUG
class Args:
    checkpoints = ['fxrZE', 'lJu2W', 'rdEIg', 'esdq2' ,'vnhKs'] # Ordered KFOLDS order. Make this nicer.
    checkpoint_fp = '/userdata/smetzger/all_deepul_files/ckpts'
    data = '/userdata/smetzger/data/cifar_10/'
    num_op = 2
    num_policy=5
    num_search = 200
    dataid = 'cifar10'
    cv_ratio=1.0
    smoke_test=True
    resume=False
    arch = 'resnet50'
    distributed=False
    workers=32
args=Args()


import random

import PIL, PIL.ImageOps, PIL.ImageEnhance, PIL.ImageDraw
import numpy as np
import torch
from torchvision.transforms.transforms import Compose
import torchvision
import torchvision.transforms as transforms
random_mirror = True


def ShearX(img, v):  # [-0.3, 0.3]
    assert -0.3 <= v <= 0.3
    if random_mirror and random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, v, 0, 0, 1, 0))


def ShearY(img, v):  # [-0.3, 0.3]
    assert -0.3 <= v <= 0.3
    if random_mirror and random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, v, 1, 0))


def TranslateX(img, v):  # [-150, 150] => percentage: [-0.45, 0.45]
    assert -0.45 <= v <= 0.45
    if random_mirror and random.random() > 0.5:
        v = -v
    v = v * img.size[0]
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, v, 0, 1, 0))


def TranslateY(img, v):  # [-150, 150] => percentage: [-0.45, 0.45]
    assert -0.45 <= v <= 0.45
    if random_mirror and random.random() > 0.5:
        v = -v
    v = v * img.size[1]
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, 0, 1, v))


def TranslateXAbs(img, v):  # [-150, 150] => percentage: [-0.45, 0.45]
    assert 0 <= v <= 10
    if random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, v, 0, 1, 0))


def TranslateYAbs(img, v):  # [-150, 150] => percentage: [-0.45, 0.45]
    assert 0 <= v <= 10
    if random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, 0, 1, v))


def Rotate(img, v):  # [-30, 30]
    assert -30 <= v <= 30
    if random_mirror and random.random() > 0.5:
        v = -v
    return img.rotate(v)


def AutoContrast(img, _):
    return PIL.ImageOps.autocontrast(img)


def Invert(img, _):
    return PIL.ImageOps.invert(img)


def Equalize(img, _):
    return PIL.ImageOps.equalize(img)


def Flip(img, _):  # not from the paper
    return PIL.ImageOps.mirror(img)


def Solarize(img, v):  # [0, 256]
    assert 0 <= v <= 256
    return PIL.ImageOps.solarize(img, v)


def Posterize(img, v):  # [4, 8]
    assert 4 <= v <= 8
    v = int(v)
    return PIL.ImageOps.posterize(img, v)


def Posterize2(img, v):  # [0, 4]
    assert 0 <= v <= 4
    v = int(v)
    return PIL.ImageOps.posterize(img, v)


def Contrast(img, v):  # [0.1,1.9]
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Contrast(img).enhance(v)


def Color(img, v):  # [0.1,1.9]
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Color(img).enhance(v)


def Brightness(img, v):  # [0.1,1.9]
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Brightness(img).enhance(v)


def Sharpness(img, v):  # [0.1,1.9]
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Sharpness(img).enhance(v)


def Cutout(img, v):  # [0, 60] => percentage: [0, 0.2]
    assert 0.0 <= v <= 0.2
    if v <= 0.:
        return img

    v = v * img.size[0]
    return CutoutAbs(img, v)


def CutoutAbs(img, v):  # [0, 60] => percentage: [0, 0.2]
    # assert 0 <= v <= 20
    if v < 0:
        return img
    w, h = img.size
    x0 = np.random.uniform(w)
    y0 = np.random.uniform(h)

    x0 = int(max(0, x0 - v / 2.))
    y0 = int(max(0, y0 - v / 2.))
    x1 = min(w, x0 + v)
    y1 = min(h, y0 + v)

    xy = (x0, y0, x1, y1)
    color = (125, 123, 114)
    # color = (0, 0, 0)
    img = img.copy()
    PIL.ImageDraw.Draw(img).rectangle(xy, color)
    return img


def SamplePairing(imgs):  # [0, 0.4]
    def f(img1, v):
        i = np.random.choice(len(imgs))
        img2 = PIL.Image.fromarray(imgs[i])
        return PIL.Image.blend(img1, img2, v)

    return f


def augment_list(for_autoaug=True):  # 16 oeprations and their ranges
    l = [
        (ShearX, -0.3, 0.3),  # 0
        (ShearY, -0.3, 0.3),  # 1
        (TranslateX, -0.45, 0.45),  # 2
        (TranslateY, -0.45, 0.45),  # 3
        (Rotate, -30, 30),  # 4
        (AutoContrast, 0, 1),  # 5
        (Invert, 0, 1),  # 6
        (Equalize, 0, 1),  # 7
        (Solarize, 0, 256),  # 8
        (Posterize, 4, 8),  # 9
        (Contrast, 0.1, 1.9),  # 10
        (Color, 0.1, 1.9),  # 11
        (Brightness, 0.1, 1.9),  # 12
        (Sharpness, 0.1, 1.9),  # 13
        (Cutout, 0, 0.2),  # 14
        # (SamplePairing(imgs), 0, 0.4),  # 15
    ]
    if for_autoaug:
        l += [
            (CutoutAbs, 0, 20),  # compatible with auto-augment
            (Posterize2, 0, 4),  # 9
            (TranslateXAbs, 0, 10),  # 9
            (TranslateYAbs, 0, 10),  # 9
        ]
    return l


augment_dict = {fn.__name__: (fn, v1, v2) for fn, v1, v2 in augment_list()}


def get_augment(name):
    return augment_dict[name]


def apply_augment(img, name, level):
    augment_fn, low, high = get_augment(name)
    return augment_fn(img.copy(), level * (high - low) + low)


class Lighting(object):
    """Lighting noise(AlexNet - style PCA - based noise)"""

    def __init__(self, alphastd, eigval, eigvec):
        self.alphastd = alphastd
        self.eigval = torch.Tensor(eigval)
        self.eigvec = torch.Tensor(eigvec)

    def __call__(self, img):
        if self.alphastd == 0:
            return img

        alpha = img.new().resize_(3).normal_(0, self.alphastd)
        rgb = self.eigvec.type_as(img).clone() \
            .mul(alpha.view(1, 3).expand(3, 3)) \
            .mul(self.eigval.view(1, 3).expand(3, 3)) \
            .sum(1).squeeze()

        return img.add(rgb.view(3, 1, 1).expand_as(img))

class Augmentation(object):
    def __init__(self, policies):
        self.policies = policies

    def __call__(self, img):
        for _ in range(1):
            policy = random.choice(self.policies)
            for name, pr, level in policy:
                if random.random() > pr:
                    continue
                img = apply_augment(img, name, level)
        return img

_CIFAR_MEAN, _CIFAR_STD = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

from collections import defaultdict
class Accumulator:
    def __init__(self):
        self.metrics = defaultdict(lambda: 0.)

    def add(self, key, value):
        self.metrics[key] += value

    def add_dict(self, dict):
        for key, value in dict.items():
            self.add(key, value)

    def __getitem__(self, item):
        return self.metrics[item]

    def __setitem__(self, key, value):
        self.metrics[key] = value

    def get_dict(self):
        return copy.deepcopy(dict(self.metrics))

    def items(self):
        return self.metrics.items()

    def __str__(self):
        return str(dict(self.metrics))

    def __truediv__(self, other):
        newone = Accumulator()
        for key, value in self.items():
            if isinstance(other, str):
                if other != key:
                    newone[key] = value / self[other]
                else:
                    newone[key] = value
            else:
                newone[key] = value / other
        return newone


print(Augmentation)

# Define how we load our dataloaders. 
_CIFAR_MEAN, _CIFAR_STD = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)


def get_dataloaders(augmentations, batch=1024, kfold=0, get_train=False):

    """
    input: augmentations: the list of the augmentations you want applied to the data. 
    batch = batchsize, 
    kfold, which fold you want to look at (0, 1,2 3, or 4)
    get_train, whether or not you want the train data. Use this when loading the data to train linear classifiers, 
    slash when you're loading the final classifier. 
    """
    if args.dataid == "imagenet":
        train_dataset = datasets.ImageFolder(
            traindir,
            transformations)

        # TODO: add imagenet transforms etc. 
    elif args.dataid == "cifar10":
        transform_train = transforms.Compose([
            transforms.RandomResizedCrop(28, scale=(0.2, 1.)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(_CIFAR_MEAN, _CIFAR_STD),
        ])

        transform_train.transforms.insert(0, Augmentation(augmentations))

        transform_test = transforms.Compose([
            transforms.Resize(32),
            transforms.CenterCrop(28),
            transforms.ToTensor(),
            transforms.Normalize(_CIFAR_MEAN, _CIFAR_STD),
        ])

    else:
        raise NotImplementedError("Support for the following dataset is not yet implemented: {}".format(args.dataid))

    if get_train: 
        train_dataset = torchvision.datasets.CIFAR10(args.data,
                                                     transform=transform_train,
                                                     download=True)

    val_dataset = torchvision.datasets.CIFAR10(args.data, transform=transform_train, 
        download=True)

    if get_train: 
        torch.manual_seed(1337)
        lengths = [len(train_dataset)//5]*5
        folds = torch.utils.data.random_split(train_dataset, lengths)
        folds.pop(kfold)
        train_dataset = torch.utils.data.ConcatDataset(folds)


    torch.manual_seed(1337)
    lengths = [len(val_dataset)//5]*5
    folds = torch.utils.data.random_split(val_dataset, lengths)
    val_dataset = folds[kfold]

    # if args.distributed:
    #     # if get_train: 
    #     #     train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    #     # val_sampler = torch.utils.data.distributed.DistributedSampler(val_dataset) #TODO: is this necessary? 
    #     val_sampler = None

    if get_train: 
        train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=batch, shuffle=(train_sampler is None),
            num_workers=args.workers, pin_memory=True, sampler=train_sampler, drop_last=True)

    val_loader= torch.utils.data.DataLoader(
        val_dataset, batch_size=batch, shuffle=False,
        num_workers=args.workers, pin_memory=True, drop_last=False
    )

    if not get_train: 
        train_loader = None

    return train_loader, val_loader

# Take in the augment from hyperopt and return some augmentations, in teh way that we want them. 
def policy_decoder(augment, num_policy, num_op):
    op_list = augment_list(False)
    policies = []
    for i in range(num_policy):
        ops = []
        for j in range(num_op):
            op_idx = augment['policy_%d_%d' % (i, j)]
            op_prob = augment['prob_%d_%d' % (i, j)]
            op_level = augment['level_%d_%d' % (i, j)]
            ops.append((op_list[op_idx][0].__name__, op_prob, op_level))
        policies.append(ops)
    return policies

def load_base_model(cv_fold): 
    
    if args.pretrained:
        if os.path.isfile(args.pretrained):
            print("=> loading checkpoint '{}'".format(args.pretrained))
            checkpoint = torch.load(args.pretrained, map_location="cpu")

            # rename moco pre-trained keys
            state_dict = checkpoint['state_dict']
            if checkpoint.get('id'):
                # sync the ids for wandb
                args.id = checkpoint['id']
                name = args.id
                wandb_resume = True
            if checkpoint.get('name'):
                name = checkpoint['name']

            for k in list(state_dict.keys()):
                # retain only encoder_q up to before the embedding layer
                if k.startswith('module.model.encoder'):
                    # remove prefix
                    state_dict[k[len("module.model.encoder."):]] = state_dict[k]
                # delete renamed or unused k
                del state_dict[k]
            args.start_epoch = 0
            msg = model.load_state_dict(state_dict, strict=False)
            assert set(msg.missing_keys) == {"fc.weight", "fc.bias"}

def load_model(cv_fold): 
    model = models.__dict__[args.arch]()
    # CIFAR 10 mod
    
    
    if args.dataid =="cifar10":
    # use the layer the SIMCLR authors used for cifar10 input conv, checked all padding/strides too.
        model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1,1), padding=(1,1), bias=False)
        model.maxpool = nn.Identity()

    # freeze all layers but the last fc
    for name, param in model.named_parameters():
        if name not in ['fc.weight', 'fc.bias']:
            param.requires_grad = False
    # init the fc layer
    if args.dataid == "cifar10":
#         print('before change', model.fc)
        model.fc = torch.nn.Linear(model.fc.in_features, 10) # note this is for cifar 10.
#         print(model.fc)


    savefile = os.path.join(args.checkpoint_fp, 
                            "{}_lincls_best.tar".format(args.checkpoints[cv_fold]))

    ckpt = torch.load(savefile, map_location="cpu")
    
    state_dict = ckpt['state_dict']
    
    for k in list(state_dict.keys()):
        # retain only encoder_q up to before the embedding layer
        
        if k.startswith('module'):
            # remove prefix
            
            state_dict[k[len("module."):]] = state_dict[k]
            del state_dict[k]
            
    
    model.load_state_dict(state_dict)
    
    return model

    # Load the FC layer and append it to the end. 

def eval_augmentations(config): 
    augment = config
    print('called', augment)
    augmentations = policy_decoder(augment, augment['num_policy'], augment['num_op'])
    # Load the model from wandb. 
    fold = augment['cv_fold']
    ckpt = args.checkpoint_fp + 'fold_%d.tar' %(fold)

    model = load_model(cv_fold).cuda()
    loaders = []
#     TODO: Undo this
    for _ in range(args.num_policy): 
    # for _ in range(2):
        _, validloader = get_dataloaders(augmentations, 128, kfold=fold)
        loaders.append(iter(validloader))
        del _

    metrics = Accumulator()
    loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
    try: 
        while True: 
            losses = []
            corrects = []

            for loader in loaders:
                data, label = next(loader)
                data = data.cuda()
                label = label.cuda()
                pred = model(data)

                loss = loss_fn(pred, label)
                losses.append(loss.detach().cpu().numpy())

                _, pred = pred.topk(1, 1, True, True)
                pred = pred.t()
                correct = pred.eq(label.view(1, -1).expand_as(pred)).detach().cpu().numpy()
                corrects.append(correct)

                del loss, correct, pred, data, label

            losses = np.concatenate(losses)
            losses_min = np.min(losses, axis=0).squeeze()

            corrects = np.concatenate(corrects)
            corrects_max = np.max(corrects, axis=0).squeeze()
            metrics.add_dict({ 
                'minus_loss': -1*np.sum(losses_min),
                'correct': np.sum(corrects_max),
                'cnt': len(corrects_max)})
            del corrects, corrects_max
    
    except StopIteration: 
        pass

    del model
    metrics = metrics/'cnt'
    # reporter(minus_loss=metrics['minus_loss'], top_1_valid=metrics['correct'], done=True)
    tune.track.log(top_1_valid=metrics['correct'])
    print(metrics['correct'])
    return metrics['correct']

ops = augment_list(False) # Get the default augmentation set. 
# Define the space of our augmentations. 
space = {}
for i in range(args.num_policy): 
    for j in range(args.num_op):
        space['policy_%d_%d' %(i,j)]  = hp.choice('policy_%d_%d' %(i, j), list(range(0, len(ops))))
        space['prob_%d_%d' %(i, j)] = hp.uniform('prob_%d_%d' %(i, j), 0.0, 1.0)
        space['level_%d_%d' %(i, j)] = hp.uniform('level_%d_%d' %(i, j), 0.0, 1.0)

final_policy_set = []
reward_attr = 'top_1_valid' # TODO: let this be whatever we want. 


ray.init(num_gpus=3, ignore_reinit_error=True)
import ray
from ray import tune

cv_num = 5
num_result_per_cv = 10

for _ in range(1): 
    for cv_fold in range(cv_num): 
        name = "search_%s_fold_%d" %(args.dataid, cv_fold)
        hyperopt_search=HyperOptSearch(space, 
            max_concurrent=3,
            metric=reward_attr,
            mode='max')


        results = tune.run(
            eval_augmentations, 
            name=name,
            num_samples=200,
            resources_per_trial={
                "gpu": 1
            },
            search_alg=hyperopt_search,
            verbose=1,
            config = { 
                'num_op': args.num_op, 
                'num_policy': args.num_policy, 
                'cv_fold': cv_fold
            },
            return_trials=True,
            stop={'training_iteration': 1},
        )
        results_copy = results
        results = [x for x in results if x.last_result is not None]
        results = sorted(results, key= lambda x: x.last_result[reward_attr], reverse=True)

        for result in results[:num_result_per_cv]: 
            final_policy = policy_decoder(result.config, args.num_policy, args.num_op)
            final_policy_set.extend(final_policy)

        print(final_policy)
print(final_policy_set)

Trial name,status,loc,level_0_0,level_0_1,level_1_0,level_1_1,level_2_0,level_2_1,level_3_0,level_3_1,level_4_0,level_4_1,policy_0_0,policy_0_1,policy_1_0,policy_1_1,policy_2_0,policy_2_1,policy_3_0,policy_3_1,policy_4_0,policy_4_1,prob_0_0,prob_0_1,prob_1_0,prob_1_1,prob_2_0,prob_2_1,prob_3_0,prob_3_1,prob_4_0,prob_4_1,iter,total time (s)
eval_augmentations_2c977ad8,TERMINATED,,0.212974,0.612565,0.64748,0.268248,0.022933,0.424442,0.854176,0.120761,0.467185,0.138461,3,2,8,14,4,13,2,1,6,14,0.109103,0.0267577,0.962301,0.304544,0.106383,0.0421813,0.403514,0.169748,0.186058,0.527183,0,31.8379
eval_augmentations_2c977ad9,TERMINATED,,0.733905,0.800095,0.971833,0.00400934,0.600714,0.989671,0.578494,0.528586,0.842865,0.431953,0,3,12,10,4,12,9,2,1,14,0.973998,0.301833,0.552866,0.0757872,0.924581,0.57,0.0687746,0.398663,0.575306,0.0694777,0,30.544
eval_augmentations_2c977ada,TERMINATED,,0.981342,0.793713,0.634592,0.274265,0.0505825,0.785674,0.611026,0.123859,0.188967,0.417366,8,2,13,4,3,14,12,2,6,8,0.737672,0.394378,0.779934,0.313188,0.0168467,0.530964,0.405915,0.982938,0.254023,0.648003,0,31.4053
eval_augmentations_2c977adb,TERMINATED,,0.910114,0.859957,0.530475,0.297999,0.638969,0.884141,0.89037,0.807023,0.154171,0.715557,6,14,9,12,0,3,8,1,11,3,0.340471,0.685007,0.00674126,0.0265304,0.910527,0.491338,0.433306,0.467246,0.84959,0.369679,0,31.6667
eval_augmentations_40595708,TERMINATED,,0.732281,0.0817008,0.337212,0.816567,0.958359,0.591864,0.776091,0.508529,0.189592,0.685458,8,8,10,8,11,2,11,8,12,14,0.171747,0.0603891,0.863233,0.607494,0.837057,0.284386,0.71623,0.810234,0.126735,0.88667,0,30.8354
eval_augmentations_40595709,TERMINATED,,0.985144,0.814745,0.650038,0.376866,0.149741,0.165238,0.488923,0.0694144,0.721777,0.715371,6,6,0,13,12,8,8,4,8,11,0.962391,0.38057,0.21726,0.801667,0.428802,0.725996,0.219158,0.488966,0.647443,0.873806,0,31.1507
eval_augmentations_4059570a,TERMINATED,,0.297082,0.413758,0.512361,0.366368,0.445189,0.204061,0.579531,0.270153,0.127178,0.290029,3,6,7,13,13,11,1,2,5,2,0.476934,0.664436,0.764752,0.260205,0.0769416,0.235244,0.318027,0.665287,0.62594,0.542851,0,32.3739
eval_augmentations_543e3b08,TERMINATED,,0.0879359,0.590835,0.450322,0.810424,0.923551,0.934888,0.0544077,0.335955,0.729825,0.427085,11,12,6,7,10,9,11,9,4,8,0.44274,0.26235,0.228615,0.547312,0.32658,0.260147,0.566597,0.618636,0.65359,0.690443,0,31.7428
eval_augmentations_543e3b09,TERMINATED,,0.56318,0.125638,0.182512,0.293829,0.743232,0.0371856,0.273345,0.85689,0.872397,0.0247433,8,12,12,13,6,11,13,1,14,0,0.926674,0.148725,0.391916,0.944113,0.176085,0.392764,0.758142,0.433691,0.0345147,0.973659,0,32.1763
eval_augmentations_543e3b0a,TERMINATED,,0.440439,0.982331,0.844836,0.380831,0.826899,0.0945822,0.219306,0.871824,0.564371,0.8382,11,4,8,7,4,3,9,13,2,8,0.363181,0.200459,0.530525,0.410972,0.378292,0.0963842,0.839745,0.345456,0.231642,0.127179,0,31.6614


[[('Posterize', 0.8067005728502419, 0.29786715629930893), ('Equalize', 0.160235053944408, 0.6723016083830553)], [('Cutout', 0.9665591104812261, 0.3735412618047689), ('Cutout', 0.0011833361791326047, 0.8931576011340583)], [('Posterize', 0.00016685514155401204, 0.017584623422554346), ('Sharpness', 0.9921423086638085, 0.528303010102314)], [('Posterize', 0.997514258460487, 0.8361471801228328), ('ShearY', 0.003881095394338452, 0.23968775715737828)], [('ShearX', 0.41400608206437645, 0.7126687730068754), ('AutoContrast', 0.6891957849457013, 0.9127219513894942)]]
[[('Solarize', 0.027166149466872055, 0.2503535572762614), ('ShearY', 0.7694060091381397, 0.5825277673496059)], [('TranslateX', 0.8555404507668116, 0.4522968261028433), ('TranslateY', 0.05464759332866556, 0.11493051049327146)], [('Sharpness', 0.31681442295103546, 0.648536762468891), ('ShearX', 0.7108630574765289, 0.6566307473142623)], [('Solarize', 0.015765553146392247, 0.9384924850465763), ('Color', 0.38922051186013157, 0.429901841524

In [2]:
# import os
# print(os.environ)
print(os.environ["CUDA_VISIBLE_DEVICES"])

# ="0,3"

KeyError: 'CUDA_VISIBLE_DEVICES'

In [ ]:
print(final_policy_set)